In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, FunctionTransformer, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.base import TransformerMixin, BaseEstimator
import random
# Класс EpsGreedy из вашего файла
class EpsGreedy:
    def __init__(self, n_arms: int, eps: float = 0.1):
        self.n_arms = n_arms
        self.eps = eps
        self.n_iters = 0
        self.arms_states = np.zeros(n_arms)
        self.arms_actions = np.zeros(n_arms)
    def flush(self):
        self.n_iters = 0
        self.arms_states = np.zeros(self.n_arms)
        self.arms_actions = np.zeros(self.n_arms)
    def update_reward(self, arm: int, reward: int):
        self.n_iters += 1
        self.arms_states[arm] += reward
        self.arms_actions[arm] += 1
    def choose_arm(self):
        if random.random() < self.eps:
            return random.randint(0, self.n_arms - 1)
        else:
            return np.argmax(self.arms_states / self.arms_actions)
        
    def get_policy(self):
        if np.all(self.arms_actions == 0):
            return np.full(self.n_arms, 1.0 / self.n_arms)
        with np.errstate(divide='ignore', invalid='ignore'):
            values = self.arms_states / self.arms_actions
        values[np.isnan(values)] = 0
        best = np.argmax(values)
        policy = np.full(self.n_arms, self.eps / self.n_arms)
        policy[best] += 1 - self.eps
        return policy 
    
class CustomFeatures(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.copy()
        if 'history' in X.columns and 'recency' in X.columns:
            X['history_recency'] = X['history'] / np.clip(X['recency'], 1, None)
        if 'mens' in X.columns and 'womens' in X.columns:
            X['mens_womens'] = X['mens'] * X['womens']
        if 'recency' in X.columns:
            X['recency_bin'] = pd.cut(X['recency'], bins=[0, 3, 6, 9, 12],
                                      labels=['very_recent', 'recent', 'medium', 'old'])
        return X
# Загрузим данные
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
# Константы
ACTION_COL = 'segment'
TARGET_COL = 'visit'
ID_COL = 'id'
ACTIONS = ['Mens E-Mail', 'Womens E-Mail', 'No E-Mail']
ACTION_TO_IDX = {a: i for i, a in enumerate(ACTIONS)}
CONTROL = 'No E-Mail'
# Препроцессинг
log_transformer = FunctionTransformer(np.log1p)
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
num_log = ['history', 'history_recency']
num_scale = ['recency']
binary = ['mens', 'womens', 'newbie', 'mens_womens']
# Убрали 'history_segment' отсюда
cat = ['zip_code', 'channel', 'recency_bin']
ct = ColumnTransformer([
    ('num_log', Pipeline([('imp', SimpleImputer(strategy='median')),
                          ('log', log_transformer),
                          ('scale', MinMaxScaler())]), num_log),
    ('num_scale', Pipeline([('imp', SimpleImputer(strategy='median')),
                            ('scale', MinMaxScaler())]), num_scale),
    ('binary', Pipeline([('imp', SimpleImputer(strategy='most_frequent')),
                         ('cast', FunctionTransformer(lambda x: x.astype(float)))]), binary),
    ('cat', Pipeline([('imp', SimpleImputer(strategy='most_frequent', fill_value='Missing')),
                      ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), cat),
], remainder='drop')
preprocess = Pipeline([
    ('custom', CustomFeatures()),
    ('ct', ct),
    ('poly', poly)
])
action_ohe = OneHotEncoder(categories=[ACTIONS], sparse_output=False, handle_unknown='ignore')
action_ohe.fit(np.array(ACTIONS).reshape(-1, 1))
# Fit preprocess на train_df без history_segment (если он есть в данных)
train_fit_df = train_df.drop(columns=['history_segment'], errors='ignore')
preprocess.fit(train_fit_df)
def build_X(df, action=None):
    # Явно удаляем history_segment, чтобы не полагаться на его наличие
    df_proc = df.drop(columns=['history_segment'], errors='ignore')
    X = preprocess.transform(df_proc)
    if action is not None:
        act = action_ohe.transform(np.full((len(df_proc), 1), action))
    else:
        act = action_ohe.transform(df_proc[[ACTION_COL]])
    return np.hstack([X, act])
def get_plog(df):
    return np.full((len(df), 3), 1.0 / 3)
def predict_all(model, df):
    probs = np.hstack([model.predict_proba(build_X(df, a))[:, 1].reshape(-1, 1) for a in ACTIONS])
    control_idx = ACTIONS.index(CONTROL)
    uplift = probs - probs[:, control_idx, None]
    return uplift
def compute_snips(df, pi):
    idx = df[ACTION_COL].map(ACTION_TO_IDX).values
    pi_ai = pi[np.arange(len(df)), idx]
    p_log = get_plog(df)[np.arange(len(df)), idx]
    r = df[TARGET_COL].values
    w = pi_ai / np.clip(p_log, 1e-8, None)
    w = np.clip(w, 0, 5)
    return np.sum(w * r) / np.sum(w)
# Фиксированный эпсилон
best_eps = 0
# Финальная модель
X_full = build_X(train_df)
y_full = train_df[TARGET_COL].values
weights_full = 1.0 / get_plog(train_df)[np.arange(len(train_df)), train_df[ACTION_COL].map(ACTION_TO_IDX).values]
estimators_final = [
    (f'lgbm_{i}', LGBMClassifier(
        n_estimators=2000,
        max_depth=5,
        learning_rate=0.01,
        subsample=0.7,
        colsample_bytree=0.6,
        reg_alpha=0.2,
        reg_lambda=0.2,
        random_state=i,
        verbose=-1
    )) for i in range(7)
]
final_model = VotingClassifier(estimators=estimators_final, voting='soft')
final_model.fit(X_full, y_full, sample_weight=weights_full)
# Вычисляем SNIPS на всей тренировочной выборке для оценки
uplift_scores = predict_all(final_model, train_df)
pi_train = np.zeros((len(train_df), 3))
for i, score in enumerate(uplift_scores):
    eg = EpsGreedy(n_arms=3, eps=best_eps)
    eg.arms_actions = np.ones(3)
    eg.arms_states = score * eg.arms_actions  # Чтобы средние были равны uplift scores
    pi_train[i] = eg.get_policy()
best_snips = compute_snips(train_df, pi_train)
print(f"\nFixed ε = {best_eps} | SNIPS on train = {best_snips:.5f}")
# Тест и сабмишн
test_uplift = predict_all(final_model, test_df)
pi_test = np.zeros((len(test_df), 3))
for i, score in enumerate(test_uplift):
    eg = EpsGreedy(n_arms=3, eps=best_eps)
    eg.arms_actions = np.ones(3)
    eg.arms_states = score * eg.arms_actions  # Чтобы средние были равны uplift scores
    pi_test[i] = eg.get_policy()
sub = pd.DataFrame({
    'id': test_df[ID_COL],
    'p_mens_email': pi_test[:, ACTION_TO_IDX['Mens E-Mail']],
    'p_womens_email': pi_test[:, ACTION_TO_IDX['Womens E-Mail']],
    'p_no_email': pi_test[:, ACTION_TO_IDX['No E-Mail']]
})
sub.iloc[:, 1:] = sub.iloc[:, 1:].clip(1e-8, 1-1e-8)
sub.iloc[:, 1:] /= sub.iloc[:, 1:].sum(axis=1).values.reshape(-1, 1)
assert np.allclose(sub.iloc[:, 1:].sum(axis=1), 1.0, atol=1e-6), "Сумма != 1"
sub.to_csv("submission_final_thanks.csv", index=False)
print("\nsubmission_uplift_epsilon_fixed_thanks2.csv готов!")
print(f"ε = {best_eps} | Ожидаемый SNIPS ≈ {best_snips:.5f}")
print(sub.head())

# **Библиотеки**

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, FunctionTransformer, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.base import TransformerMixin, BaseEstimator
import random

# **Стыренные из семинаров функции**

In [ ]:
class EpsGreedy:
    def __init__(self, n_arms: int, eps: float = 0.1):
        self.n_arms = n_arms
        self.eps = eps
        self.n_iters = 0
        self.arms_states = np.zeros(n_arms)
        self.arms_actions = np.zeros(n_arms)
    def flush(self):
        self.n_iters = 0
        self.arms_states = np.zeros(self.n_arms)
        self.arms_actions = np.zeros(self.n_arms)
    def update_reward(self, arm: int, reward: int):
        self.n_iters += 1
        self.arms_states[arm] += reward
        self.arms_actions[arm] += 1
    def choose_arm(self):
        if random.random() < self.eps:
            return random.randint(0, self.n_arms - 1)
        else:
            return np.argmax(self.arms_states / self.arms_actions)
        
    def get_policy(self):
        if np.all(self.arms_actions == 0):
            return np.full(self.n_arms, 1.0 / self.n_arms)
        with np.errstate(divide='ignore', invalid='ignore'):
            values = self.arms_states / self.arms_actions
        values[np.isnan(values)] = 0
        best = np.argmax(values)
        policy = np.full(self.n_arms, self.eps / self.n_arms)
        policy[best] += 1 - self.eps
        return policy 
    

# **Загрузка данных**

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# **Препроцессинг**

In [ ]:
class CustomFeatures(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.copy()
        if 'history' in X.columns and 'recency' in X.columns:
            X['history_recency'] = X['history'] / np.clip(X['recency'], 1, None)
        if 'mens' in X.columns and 'womens' in X.columns:
            X['mens_womens'] = X['mens'] * X['womens']
        if 'recency' in X.columns:
            X['recency_bin'] = pd.cut(X['recency'], bins=[0, 3, 6, 9, 12],
                                      labels=['very_recent', 'recent', 'medium', 'old'])
        return X


ACTION_COL = 'segment'
TARGET_COL = 'visit'
ID_COL = 'id'
ACTIONS = ['Mens E-Mail', 'Womens E-Mail', 'No E-Mail']
ACTION_TO_IDX = {a: i for i, a in enumerate(ACTIONS)}
CONTROL = 'No E-Mail'

# Препроцессинг
log_transformer = FunctionTransformer(np.log1p)
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
num_log = ['history', 'history_recency']
num_scale = ['recency']
binary = ['mens', 'womens', 'newbie', 'mens_womens']

cat = ['zip_code', 'channel', 'recency_bin']
ct = ColumnTransformer([
    ('num_log', Pipeline([('imp', SimpleImputer(strategy='median')),
                          ('log', log_transformer),
                          ('scale', MinMaxScaler())]), num_log),
    ('num_scale', Pipeline([('imp', SimpleImputer(strategy='median')),
                            ('scale', MinMaxScaler())]), num_scale),
    ('binary', Pipeline([('imp', SimpleImputer(strategy='most_frequent')),
                         ('cast', FunctionTransformer(lambda x: x.astype(float)))]), binary),
    ('cat', Pipeline([('imp', SimpleImputer(strategy='most_frequent', fill_value='Missing')),
                      ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), cat),
], remainder='drop')
preprocess = Pipeline([
    ('custom', CustomFeatures()),
    ('ct', ct),
    ('poly', poly)
])
action_ohe = OneHotEncoder(categories=[ACTIONS], sparse_output=False, handle_unknown='ignore')
action_ohe.fit(np.array(ACTIONS).reshape(-1, 1))

# **Финальная модель**

In [ ]:
train_fit_df = train_df.drop(columns=['history_segment'], errors='ignore')
preprocess.fit(train_fit_df)
def build_X(df, action=None):
    # Явно удаляем history_segment, чтобы не полагаться на его наличие
    df_proc = df.drop(columns=['history_segment'], errors='ignore')
    X = preprocess.transform(df_proc)
    if action is not None:
        act = action_ohe.transform(np.full((len(df_proc), 1), action))
    else:
        act = action_ohe.transform(df_proc[[ACTION_COL]])
    return np.hstack([X, act])
def get_plog(df):
    return np.full((len(df), 3), 1.0 / 3)
def predict_all(model, df):
    probs = np.hstack([model.predict_proba(build_X(df, a))[:, 1].reshape(-1, 1) for a in ACTIONS])
    control_idx = ACTIONS.index(CONTROL)
    uplift = probs - probs[:, control_idx, None]
    return uplift
def compute_snips(df, pi):
    idx = df[ACTION_COL].map(ACTION_TO_IDX).values
    pi_ai = pi[np.arange(len(df)), idx]
    p_log = get_plog(df)[np.arange(len(df)), idx]
    r = df[TARGET_COL].values
    w = pi_ai / np.clip(p_log, 1e-8, None)
    w = np.clip(w, 0, 5)
    return np.sum(w * r) / np.sum(w)
# Фиксированный эпсилон
best_eps = 0
# Финальная модель
X_full = build_X(train_df)
y_full = train_df[TARGET_COL].values
weights_full = 1.0 / get_plog(train_df)[np.arange(len(train_df)), train_df[ACTION_COL].map(ACTION_TO_IDX).values]
estimators_final = [
    (f'lgbm_{i}', LGBMClassifier(
        n_estimators=2000,
        max_depth=5,
        learning_rate=0.01,
        subsample=0.7,
        colsample_bytree=0.6,
        reg_alpha=0.2,
        reg_lambda=0.2,
        random_state=i,
        verbose=-1
    )) for i in range(7)
]
final_model = VotingClassifier(estimators=estimators_final, voting='soft')
final_model.fit(X_full, y_full, sample_weight=weights_full)
# Вычисляем SNIPS на всей тренировочной выборке для оценки
uplift_scores = predict_all(final_model, train_df)
pi_train = np.zeros((len(train_df), 3))
for i, score in enumerate(uplift_scores):
    eg = EpsGreedy(n_arms=3, eps=best_eps)
    eg.arms_actions = np.ones(3)
    eg.arms_states = score * eg.arms_actions  # Чтобы средние были равны uplift scores
    pi_train[i] = eg.get_policy()
best_snips = compute_snips(train_df, pi_train)
print(f"\nFixed ε = {best_eps} | SNIPS on train = {best_snips:.5f}")
# Тест и сабмишн
test_uplift = predict_all(final_model, test_df)
pi_test = np.zeros((len(test_df), 3))
for i, score in enumerate(test_uplift):
    eg = EpsGreedy(n_arms=3, eps=best_eps)
    eg.arms_actions = np.ones(3)
    eg.arms_states = score * eg.arms_actions  # Чтобы средние были равны uplift scores
    pi_test[i] = eg.get_policy()
sub = pd.DataFrame({
    'id': test_df[ID_COL],
    'p_mens_email': pi_test[:, ACTION_TO_IDX['Mens E-Mail']],
    'p_womens_email': pi_test[:, ACTION_TO_IDX['Womens E-Mail']],
    'p_no_email': pi_test[:, ACTION_TO_IDX['No E-Mail']]
})
sub.iloc[:, 1:] = sub.iloc[:, 1:].clip(1e-8, 1-1e-8)
sub.iloc[:, 1:] /= sub.iloc[:, 1:].sum(axis=1).values.reshape(-1, 1)
assert np.allclose(sub.iloc[:, 1:].sum(axis=1), 1.0, atol=1e-6), "Сумма != 1"
sub.to_csv("submission_final_thanks.csv", index=False)
print("\nsubmission_uplift_epsilon_fixed_thanks2.csv готов!")
print(f"ε = {best_eps} | Ожидаемый SNIPS ≈ {best_snips:.5f}")
print(sub.head())